In [ ]:
#Word Embeddings
in word embedding dimension is in between 50 and 300 
if it is greater we reduce it in 2D or 3D (we use PCA library)
word to vector in 2013
This enables machines to understand semantic relationships between words.
    
1.continuous BOW (big of word)
    Goal: Predict a target word based on its context (surrounding words).
Sentence: "The cat sits on the mat."
Context: ["The", "sits", "on", "the"] → Target: "cat"
better for smaller data set
fast
#architecture
input layer (context word) (one hot vector)
hidden layer (progection layer) 
output layer (probability vector through softmax function)
The model takes the surrounding words as input and tries to guess the center word.

2.Continuous Skip-gram
    Goal: Predict the context words given a target word.
    Sentence: "The cat sits on the mat."
Target: "cat" → Context: ["The", "sits", "on", "the"]
The model takes a single word as input and tries to predict words around it
preffered for larger dataset
slow



In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Sample data
text = "the quick brown fox jumps over the lazy dog".split()

# Create vocabulary
vocab = set(text)
word2idx = {word: i for i, word in enumerate(vocab)}
idx2word = {i: word for word, i in word2idx.items()}
vocab_size = len(vocab)

# Generate CBOW training pairs
def generate_cbow_pairs(text, window_size=2):
    pairs = []
    for i in range(window_size, len(text) - window_size):
        context = [text[i - j - 1] for j in range(window_size)] + \
                  [text[i + j + 1] for j in range(window_size)]
        target = text[i]
        pairs.append((context, target))
    return pairs

cbow_pairs = generate_cbow_pairs(text)

# One-hot encoding
def one_hot(index):
    vec = np.zeros(vocab_size)
    vec[index] = 1
    return vec

# Prepare input and output tensors
X = []
Y = []
for context, target in cbow_pairs:
    context_vec = np.sum([one_hot(word2idx[w]) for w in context], axis=0)
    X.append(context_vec)
    Y.append(word2idx[target])

X = torch.Tensor(X)
Y = torch.LongTensor(Y)

# CBOW Model Definition
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Linear(vocab_size, embedding_dim)
        self.output = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        x = self.embeddings(x)
        x = self.output(x)
        return x

# Model, Loss, Optimizer
model = CBOW(vocab_size, 10)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training Loop
for epoch in range(100):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_fn(output, Y)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: 2.1472
Epoch 10, Loss: 2.0958
Epoch 20, Loss: 2.0473
Epoch 30, Loss: 2.0012
Epoch 40, Loss: 1.9571
Epoch 50, Loss: 1.9145
Epoch 60, Loss: 1.8732
Epoch 70, Loss: 1.8329
Epoch 80, Loss: 1.7933
Epoch 90, Loss: 1.7544


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Sample data
text = "the quick brown fox jumps over the lazy dog".split()

# Create vocabulary
vocab = set(text)
word2idx = {word: i for i, word in enumerate(vocab)}
idx2word = {i: word for word, i in word2idx.items()}
vocab_size = len(vocab)

# Generate CBOW training pairs
def generate_cbow_pairs(text, window_size=2):
    pairs = []
    for i in range(window_size, len(text) - window_size):
        context = [text[i - j - 1] for j in range(window_size)] + \
                  [text[i + j + 1] for j in range(window_size)]
        target = text[i]
        pairs.append((context, target))
    return pairs

cbow_pairs = generate_cbow_pairs(text)

# One-hot encoding
def one_hot(index):
    vec = np.zeros(vocab_size)
    vec[index] = 1
    return vec

# Prepare input and output tensors
X = []
Y = []
for context, target in cbow_pairs:
    context_vec = np.sum([one_hot(word2idx[w]) for w in context], axis=0)
    X.append(context_vec)
    Y.append(word2idx[target])

X = torch.Tensor(X)
Y = torch.LongTensor(Y)

# CBOW Model Definition
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Linear(vocab_size, embedding_dim)
        self.output = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        x = self.embeddings(x)
        x = self.output(x)
        return x

# Model, Loss, Optimizer
model = CBOW(vocab_size, 10)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training Loop
for epoch in range(100):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_fn(output, Y)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Prediction after training
test_context = ["the", "quick", "fox", "jumps"]  # context for predicting "brown"
context_vec = np.sum([one_hot(word2idx[w]) for w in test_context], axis=0)
context_tensor = torch.Tensor([context_vec])  # batch dimension

with torch.no_grad():
    output = model(context_tensor)
    predicted_idx = torch.argmax(output, dim=1).item()
    predicted_word = idx2word[predicted_idx]

print(f"\nPredicted center word for context {test_context}: '{predicted_word}'")


Epoch 0, Loss: 2.2462
Epoch 10, Loss: 2.1799
Epoch 20, Loss: 2.1206
Epoch 30, Loss: 2.0664
Epoch 40, Loss: 2.0163
Epoch 50, Loss: 1.9694
Epoch 60, Loss: 1.9250
Epoch 70, Loss: 1.8826
Epoch 80, Loss: 1.8420
Epoch 90, Loss: 1.8028

Predicted center word for context ['the', 'quick', 'fox', 'jumps']: 'brown'


In [ ]:
#assignment

Build a simple CBOW model to learn word embeddings using a small corpus. You'll:

Create a vocabulary

Generate context-target pairs

One-hot encode words

Train a shallow neural network with one hidden layer to predict the center word from surrounding context words.

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 1. Sample corpus
corpus ="I love playing football because football is a great sport and playing it makes me happy".lower().split()

# 2. Create vocabulary
vocab = sorted(set(corpus))
word2idx = {word: i for i, word in enumerate(vocab)}
idx2word = {i: word for word, i in word2idx.items()}
vocab_size = len(vocab)

# 3. Generate context-target pairs (window size = 2)
def generate_cbow_data(text, window_size=2):
    pairs = []
    for i in range(window_size, len(text) - window_size):
        context = [text[i - j - 1] for j in range(window_size)] + \
                  [text[i + j + 1] for j in range(window_size)]
        target = text[i]
        pairs.append((context, target))
    return pairs

cbow_pairs = generate_cbow_data(corpus)

# 4. One-hot encode context words and prepare input/output tensors
def one_hot_vector(index, vocab_size):
    vec = np.zeros(vocab_size)
    vec[index] = 1
    return vec

X = []
Y = []
for context, target in cbow_pairs:
    context_vec = np.sum([one_hot_vector(word2idx[word], vocab_size) for word in context], axis=0)
    X.append(context_vec)
    Y.append(word2idx[target])

X = torch.Tensor(X)
Y = torch.LongTensor(Y)

# 5. Define CBOW model (shallow NN with one hidden layer)
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.hidden = nn.Linear(vocab_size, embedding_dim)
        self.output = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        x = self.hidden(x)
        x = torch.relu(x)  # Activation in hidden layer
        x = self.output(x)
        return x

# 6. Initialize model, loss, optimizer
embedding_dim = 10
model = CBOW(vocab_size, embedding_dim)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 7. Training loop
for epoch in range(200):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_fn(output, Y)
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# 8. Visualize embeddings (optional)
embeddings = model.hidden.weight.data
print("\nLearned word embeddings:")
for word, idx in word2idx.items():
    print(f"{word:>5}: {embeddings[:, idx].detach().numpy()}")


Epoch 0, Loss: 2.6582
Epoch 20, Loss: 1.7392
Epoch 40, Loss: 0.7359
Epoch 60, Loss: 0.2351
Epoch 80, Loss: 0.0838
Epoch 100, Loss: 0.0404
Epoch 120, Loss: 0.0248
Epoch 140, Loss: 0.0174
Epoch 160, Loss: 0.0131
Epoch 180, Loss: 0.0103

Learned word embeddings:
    a: [ 0.97888726  0.88240165  0.973537    0.20303002 -0.05467448  0.30904806
  0.29014635 -0.46173197  0.13534695 -0.7366295 ]
  and: [-0.01963021  0.4019308   0.7408129   0.9595285  -0.13770202  0.46147734
 -0.07047568  0.23587021 -0.81130284  0.07828525]
because: [ 0.9554598   0.5189271  -0.8518682   0.7884116  -0.25491792 -0.32067814
 -1.1183769  -0.7293538   0.5239446  -0.5143575 ]
football: [ 0.01161816 -0.17910893 -0.93798995  0.91277957 -0.05154774  0.6004798
  0.80328107 -0.01622613  0.8789514  -0.70501494]
great: [ 0.8490904   0.8564599   0.44155273 -1.067832    0.17103451 -0.8941354
  1.2274886   0.8291184   0.6733979   0.10236617]
happy: [-0.2348434  -0.17415917 -0.9824613  -0.42830747  0.07543609 -0.57974344
 -0.284